In [3]:
import utils.extract_df as extract_df
import utils.transform as transform 
import utils.clustering as clustering 
import utils.main as main
import streamlit as st
import pandas as pd

In [19]:
df = extract_df.readcsv('data/taxi_data.csv')

In [5]:
extract_df.readshp('data/nyc-boundaries/geo_export_9ca5396d-336c-47af-9742-ab30cd995e41.shp')

## Cleaning locations 

In [14]:
import geopandas as gpd

# Read the shapefile of NYC
nyc = gpd.read_file('data/nyc-boundaries/geo_export_9ca5396d-336c-47af-9742-ab30cd995e41.shp')

In [22]:
### pick up missings
missingPickup_coords = (df['pickup_longitude'] == 0) | (df['pickup_longitude'] > -72) | (df['pickup_latitude'] == 0)
num_missingPickup = sum(missingPickup_coords)
num_missingPickup

1653

In [26]:
# Generate random points within the NYC polygon
random_pickup_points = nyc.geometry.sample_points(num_missingPickup, random_state=42)
random_pickup_points

0    MULTIPOINT (-74.25405 40.50616, -74.25214 40.5...
1    MULTIPOINT (-74.04602 40.69064, -74.04205 40.6...
2    MULTIPOINT (-73.93318 40.83458, -73.93283 40.8...
3    MULTIPOINT (-74.04162 40.62463, -74.03999 40.6...
4    MULTIPOINT (-73.96147 40.73956, -73.96079 40.7...
Name: sampled_points, dtype: geometry

In [6]:
df_transform = transform.dataTransformation(df = extract_df.readcsv('data/taxi_data.csv'), 
nyc = extract_df.readshp('data/nyc-boundaries/geo_export_9ca5396d-336c-47af-9742-ab30cd995e41.shp'))

In [7]:
df_transform = df_transform.transform()

In [8]:
(pd.to_datetime(df_transform.pickup_datetime.min()), pd.to_datetime(df_transform.pickup_datetime.max()))

(Timestamp('2014-01-01 00:24:00+0000', tz='UTC'),
 Timestamp('2015-06-30 23:57:04+0000', tz='UTC'))

In [9]:
df_transform.columns

Index(['dropoff_latitude', 'dropoff_longitude', 'fare_amount', 'feat01',
       'feat02', 'feat03', 'feat04', 'feat05', 'feat06', 'feat07', 'feat08',
       'feat09', 'feat10', 'passenger_count', 'pickup_datetime',
       'pickup_latitude', 'pickup_longitude', 'passenger_big_group',
       'fare_amount_log', 'year', 'month', 'day', 'hour', 'trip_distance'],
      dtype='object')

## Clustering step 

In [10]:
df_clustered = clustering.pickUpCluster(df_transform).clusterCreated()

In [11]:
df_clustered.head()

,dropoff_latitude,dropoff_longitude,fare_amount,feat01,feat02,feat03,feat04,feat05,feat06,feat07,...,pickup_latitude,pickup_longitude,passenger_big_group,fare_amount_log,year,month,day,hour,trip_distance,pickup_cluster
0,40.768550,-73.862065,52.713,0.607633,0.680994,0.869333,0.359081,0.283538,0.898003,0.481185,...,40.759570,-73.985715,1,3.964862,2014,1,8,6,10.461512,0
1,40.746906,-73.990494,19.350,0.353808,0.555256,0.946294,0.530530,0.453938,0.708570,0.161038,...,40.759457,-73.972038,0,2.962692,2015,2,16,20,2.089068,0
2,40.697496,-73.984946,24.850,0.248761,0.271752,0.418165,0.368993,0.362234,0.257532,0.710595,...,40.705860,-74.013626,0,3.212858,2014,3,18,13,2.590397,3
3,40.767617,-73.959482,16.600,0.606718,0.809065,0.826723,0.228102,0.819767,0.859372,0.014095,...,40.779830,-73.955313,0,2.809403,2014,3,20,18,1.402666,2
4,40.726230,-73.983539,14.400,0.535181,0.873709,0.757857,0.263007,0.711840,0.635563,0.126834,...,40.688690,-73.805695,0,2.667228,2014,5,6,23,15.560976,1


## Run the application

In [12]:
! streamlit run Homepage.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.13:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
/Users/irenaz/.pyenv/versions/3.9.18/lib/python3.9/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
^C
  Stopping...
